
## Analisis Red Social

In [1]:
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import sys
import pandas as pd

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.gridspec import GridSpec
#import missingno as msno
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import string
import sys
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score,roc_curve
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import time
import sys
import math
import itertools
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#import folium
#from folium.plugins import Fullscreen
#import geopandas as gpd
import json
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')
stop = stopwords.words('portuguese')
#stop.drop('nao') #Stopword already have "Não", just adding this because it's appear on dataframe

## Aca leo la base de datos a usar con las variables

In [3]:
os.chdir("C:\\Users\\d.cadavid141\\Documents\\Coisas Daniela\\201920\\201920\\00_Bases\\brazilian-ecommerce\\")
#os.getcwd()
orders=pd.read_csv('data_tratada_v_nao.csv')


####### ACA SE CREA EL BOW
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(orders.review_comment_message).toarray()


In [4]:
cols=vectorizer.get_feature_names()

In [5]:
####### ACA SE CREA EL TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
orders_redes=pd.read_csv('data_tratada_manual_grande.csv')
X_tfidf = vectorizer.fit_transform(orders_redes.Comment).toarray()
X_val_mod=pd.DataFrame(X_tfidf)
y_val=orders_redes.Label
cols_manual=vectorizer.get_feature_names()
X_val_mod.columns = cols_manual
X_val_mod.head()

,abafar,abaixar,abrir,absurdo,acabamento,acabar,aceitar,acender,acessorio,acessorios,...,voltar,volumar,voz,whatsapp,xiaomi,ximo,youtube,zap,zero,zumbir
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Genera datasets

In [6]:
def generate_zeros_dataset(orig_df, other_df):
    rows = other_df.shape[0]
    orig_colnames = list(orig_df.columns)
    dtypes = dict(orig_df.dtypes)
    d = pd.DataFrame(np.zeros((rows, len(orig_colnames))), columns=orig_colnames)
    return d.astype(dtypes) # force the same datatypes =-)

# grab both dataset columns
def generate_dataset(orig_df, other_df):
    colnames_2 = list(other_df.columns)
    d = generate_zeros_dataset(orig_df, other_df)
    d[colnames_2] = other_df.copy()
    return d

In [7]:
X_val_mod=X_val_mod[X_val_mod.columns.intersection(cols)]
rows=X_val_mod.shape[0]

In [8]:
data_orig=pd.DataFrame(np.zeros((rows,len(cols))),columns=cols)

In [9]:
base_original_tratar = generate_dataset(data_orig,X_val_mod)

In [10]:
# Standardize the data to have a mean of ~0 and a variance of 1
X_std = StandardScaler().fit_transform(base_original_tratar)


In [11]:
# load the model from disk
filename='finalized_model_pca.sav'
pca = pickle.load(open(filename, 'rb'))

In [12]:
X_val_pca=pca.transform(X_std)

In [13]:
X_val_mod=pd.DataFrame(X_val_pca)

In [14]:
X_val_mod.shape

(842, 3216)

## Boosting

In [15]:
vars_usar=245

In [16]:
reg_manual=X_val_mod.iloc[:,0:vars_usar]
reg_manual.shape

(842, 245)

In [17]:
# load the model from disk
filename='finalized_model_pca_gb.sav'
loaded_model = pickle.load(open(filename, 'rb'))
gb_roc_auc=roc_auc_score(y_val, loaded_model.predict(reg_manual))

In [18]:
gb_roc_auc

0.8306585500830105

In [19]:
gb_f1_test = f1_score(y_val, loaded_model.predict(reg_manual))
gb_f1_test

0.887298747763864

## Reg Logistica 

In [20]:
vars_usar=2849

In [21]:
lr_manual = X_val_mod.iloc[:,0:vars_usar]

In [22]:
# load the model from disk
filename='finalized_model_pca_lr.sav'
loaded_model = pickle.load(open(filename, 'rb'))
lr_roc_auc=roc_auc_score(y_val, loaded_model.predict(lr_manual))

In [23]:
lr_roc_auc

0.8488894199325854

In [24]:
lr_f1_test = f1_score(y_val, loaded_model.predict(lr_manual))
lr_f1_test

0.8927272727272728

## Random Forest

In [25]:
vars_usar=20

In [26]:
rf_manual = X_val_mod.iloc[:,0:vars_usar]

In [27]:
# load the model from disk
filename='finalized_model_pca_rf.sav'
loaded_model = pickle.load(open(filename, 'rb'))
rf_roc_auc=roc_auc_score(y_val, loaded_model.predict(rf_manual))

In [28]:
rf_roc_auc

0.8108680887457865

In [29]:
rf_f1_test = f1_score(y_val, loaded_model.predict(rf_manual))
rf_f1_test

0.841802492809204